In [ ]:
!pip install pymysql> /dev/null

In [ ]:
import json
import pymysql

In [ ]:
conn = pymysql.connect(
    host='132.226.232.76',
    user='test5',
    password='Multicampus2021!',
    database ='mcdb5',
    port=3306
)

In [ ]:
import pymysql
conn = pymysql.connect(
    host=config['host'], 
    user=config['user'], 
    password=config['password'], 
    database=config['database'], 
    port=config['port']
)

# 패스워드 암호화
* Secure Hash Algorithm(SHA-256)

In [ ]:
import hashlib

pwd='1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

* Base64로 인코딩

In [ ]:
import base64

base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [ ]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [ ]:
len(hashed_pwd)

44

* 패스워드를 생성하는 함수

In [ ]:
import hashlib, base64

def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd


In [ ]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## users table에 패스워드 넣기
* uid +21

In [ ]:
sql = "select uid from users;"
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('jbpark',),
 ('wjlee',),
 ('yunao',),
 ('yurilee',))

In [ ]:
sql_update ="update users set pwd=%s where uid=%s;"
for line in results:
  uid = line[0]
  pwd= gen_pwd(uid+'21')
  cur.execute(sql_update,(pwd, uid))
conn.commit()

* email 추가 : uid@mc.com 

In [ ]:
sql_email ="update users set email=%s where uid=%s;"

for line in results:
  uid = line[0]
  email= uid + '@mc.com'
  cur.execute(sql_email,(email, uid))
conn.commit()

### 로그인
* Case 1: 아이디가 없는 경우

In [ ]:
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'

In [ ]:
# result = Null
uid ='fool'
cur.execute(sql_login,(uid,))
result = cur.fetchone()
result

* Case 2: 비밀번호가 다른 경우

In [ ]:
uid,pwd ='djy','djy23'
cur.execute(sql_login,(uid,))
result=cur.fetchone()
result

('djy', 'mxsnWYLpB/nAOt37fpt3PMXpGGQprktT3vm2p9RfFyY=')

In [ ]:
db_pwd= result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
  print('Login')
else:
  print('Incorrect password!')

Incorrect password!


* Case 3: OK

In [ ]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Login


* 3가지 경우를 합침

In [ ]:
uid, pwd = 'djy1', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
else:
    print('Bad uid!!!')

Bad uid!!!


* 예외처리를 통해 해결하는 경우

In [ ]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
except:
    print('Bad uid!!!')

Login


### 마무리

In [ ]:
cur.close()
conn.close()